<a href="https://colab.research.google.com/github/kocurvik/edu/blob/master/PV/prezentacie/cv11.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **11. cvičenie** - Deep Learning

V tomto cvičení si ukážeme ako použiť hlbokú konvolučnú neurónovú sieť na detekciu objektov.

Najprv si však prejdeme základy používania Google Colabu, NumPy a OpenCV.

## Google Colab

Google Colab (http://colab.research.google.com) je nespoplatnená služba na ktorej je možné spúštať tzv. ipython noteboky. Služba poskytuje k dispozícii aj výpočty na GPU a CPU. Pre využitie tejto služby je nutné mať Google účet. Tieto notebooky môžeme spúšťať aj lokálne pomocou príkazu `jupyter notebook` v zložke kde máme uložené ipynb súbory.

V notebookoch máme dva druhy buniek. Textové, ako napr. tento a kód. Bunky s kódom môžeme spúštať tlačidlom play naľavo od bunky, alebo v prostredí Colab naraz od prvej až po poslednú výberom Runtime -> Run All. Defaultne notebook nemá pripojené GPU, to môžeme zmeniť zavolaním Runtime -> Change runtime type. Bunky majú spoločný scope ale je treba dávať pozor na poradie spúštania jednotlivých buniek. Preto je často vhodné použiť Runtime -> Restart and run all.

V nasledujúcej bunke si môžete otestovať spustenie python kódu.

In [ ]:
a = [5,'Hello']
s = '{} World!'
print(s.format(a[1]))

def najlepsia_funkcia(arg):
  return arg + 5

print(najlepsia_funkcia(8))

Keď sme v Google Colab chceli by sme mať prístup k súborom. To je najjednoduchšie pomocou pripojenia Google Drive k našej inštancii. To sa robí následovne:
(V Google Drive si vyrobte zložku Colab, alebo upravte na drive)

In [ ]:
from google.colab import drive
drive.mount('/content/gdrive')
root_path = 'gdrive/My Drive/Colab/'

Môžeme potom k zložke pristupovať.

In [ ]:
import os
os.listdir(root_path)

Takisto môžeme volať unixové príkazy pomocou výkričníka.

In [ ]:
!pwd
!ls
!mkdir random_dir
!ls
!wget https://www.google.com/images/branding/googlelogo/1x/googlelogo_color_272x92dp.png
!ls

Najpohodlnejšia možnosť je použiť inferface naľovo a pozrieť si záložku Files

**<----**

## NumPy
Pri práci s neurónovými sieťami je základnou štruktúrou tzv. tenzor. Tenzor je zobecnenie vektorov a matíc. Vektor je tenzor prvého rádu, matica je tenzor druhého rádu. Tenzor tretieho rádu vieme indexovať navyše ešte jedným indexom. Niekedy sa za rád tensoru označuje aj dimenzia, to je ale mätúce, ale väčšinou je to z kontextu zrejmé. Správne napríklad tenzor rádu 3 má dimenzie 20 x 24 x 6.

Podľa konvencie numpy väčšinou importuje ako np. Vektor, teda tenzor prvého rádu, vieme vyrobiť z jednuchého listu. Pomocou np.array. Často je dôležité použiť správny dátový typ to urobíme pomocou argumentu dtype. **Vynechanie tejto časti môže viesť na rôzne komplikácie!**

In [ ]:
import numpy as np

a = np.array([3, 2, 3, 4])
print(a)
b = np.array([1,5,7], dtype=np.float32)
print(b)

Maticu vieme vyrobiť z listu listov, ale treba dať pozor aby nám sedeli dĺžky vnútorných listov. Inak sa vytvorí tenzor prvého rádu, ktorý obsahuje listy.

In [ ]:
A = np.array([[1,5,8],[50,60,84]])
print(A)
B = np.array([[7,8],[6,7],[0,9,4]])
print(B)

Objekt array má metódu shape, ktorá nám umožňuje zistiť jej tvar (teda dimenzie), metódu dtype ktorá vracia typ vnútorných objektov a metódu astype, ktorá vráti tenzor v podľa špecifikovaného typu.

In [ ]:
print(a.shape)
print(A.shape)
print(A.dtype)
C = A.astype(np.float32)
print(C.dtype)


Numpy má aj funkcie na tvorenie základných matíc. Tieto príkazy sú np.zeros, np.ones a np.empty.

In [ ]:
z = np.zeros([5,10])
print(z)
o = np.ones([3,4,5])
print(o)
e = np.empty([6])
print(e)

Je možné tvoriť aj náhodné čísla pomocou np.random.random

In [ ]:
r = np.random.random([6,10,3])
print(r)

Tieto náhodné čísla použijeme na prácu s indexmi. Indexácia je podobná ako v matlabe, ale platia tu pythonovské konvencie. Indexuje sa viacerými indexmi. Ak sa indexuje menej indexmi, tak sa tie ostatné berú ako :, čo znamená zobartie všetkých prvkov pozdĺž dimenzie.

In [ ]:
print(r[3,4,1])
print(r[:,:,-1])
print(r[:,:,1].shape)
print(r[0:4,5:6,:])
print(r[0:4,5:6,:].shape)

Indexovať môžeme aj s preskakovaním jednotlivých prvkov, tzv. slicing. To sa robí pomocou ďalšej dvojbodky. Formát je potom \[začiatok:koniec:skok\]. Ak niečo z toho vynecháme, tak žačiatok bude defaultne 0, koniec -1 a skok 1.

In [ ]:
p = np.arange(25)
print(p)
print(p[4:16:2])
print(p[2:-4:6])
print(p[:10:])
print(p[::3])
print(p[1::6])
print(p[-6:])
print(r[1::2,0::3,:])

Pri deep learningu často potrebujeme vytvoriť tzv. singleton dimenziu. To sa dá urobiť pomocou pridania None indexu, alebo viac pythonic spôsobom pomocou np.newaxis.

In [ ]:
print(r[None,:,:,:].shape)
print(r[None].shape)
print(r[np.newaxis,:,:,:].shape)
print(r[:,:,:,np.newaxis].shape)
print(r[np.newaxis,:,0,:].shape)

Podobne ako v matlabe existuje broadcasting.

In [ ]:
r += 10
print(r)
r[0,:,:] = np.random.random([10,3])
print(r.shape)
r[0] = np.random.random([10,3])
print(r.shape)
r[0] = np.zeros([10,1])
print(r.shape)
r/=500
print(r)

Je možné polia pretvarovať pomocou np.reshape(arr, shape)

In [ ]:
q = np.reshape(r, [6,30])
print(q.shape)

Indexovať môžeme aj pomocou podmienok.

In [ ]:
r = np.random.random([5,5])
print(r)
r[r < 0.5] = -500
print(r)

Podmienky môžeme použiť aj np.where(cond, a1, a2), ktorá vráti pole ktoré ma na miestach kde je cond splnená hodnoty z a1 a inde hodnoty z a2. Použijeme tu príkaz np.arange(i), ktorý vráti pole podobne ako range z pythonu.

In [ ]:
a = np.arange(10)
print(a)
b = np.where(a < 5, a, a**2)
print(b)

## OpenCV

OpenCV je knižnica na manipuláciu obrazu. Na predmetoch spracovania obrazu a počítačového videnia používame Matlab na prácu s obrazom. V pythone môžeme použiť na väčšinu týchto operácií OpenCV. Ak by ste chceli použiť OpenCV inde ako v Google Colab môžete si ho nainštalovať cez pip ako balík opencv-python.

In [ ]:
import cv2

Načítanie obrazu je v OpenCV jednoduché. Načítame si obrázok, ktorý sme stiahli cez v úvodnej časti cvičenia. Vidíme, že tento obrázok je NumPy pole. S rozmermi šírka x výška x 3. Posledný kanál reprezentuje tri RGB zložky. Máme teda tri matice a z nich každá intenzitu jednej z troch zložiek. Je treba si uvedomiť, že **OpenCV defaultne pracuje s obrazom v poradí BGR!**

In [ ]:
img = cv2.imread('googlelogo_color_272x92dp.png')
print(type(img))
print(img.shape)
print(img.dtype)

Zobrazovať obrázok v OpenCV je trocha komplikované. Keďže sme v notebooku potrebujeme na to inú knižnicu. Konkrétne matplotlib. Pri zobrazovaní obrazu je nutné prekonvertovať obraz z BGR na RGB.


Zakomentovaná časť kódu ukazuje ako sa to robí pri spúštaní mimo notebooku. cv2.waitKey zastaví beh programu napočet milisekúnd v jej argumente, alebo dokedy je stlačená klávesa. V prípade, že argument je 0, tak čaká na klávesu donekončna.

In [ ]:
import matplotlib.pyplot as plt
plt.imshow(img[:,:,::-1])
plt.show()

# cv2.imshow('Obrazok', img)
# if cv2.waitKey(0) & 0xFF == ord('q'):
#     return

Môžeme zobraziť aj jednotlivé kanály.

In [ ]:
plt.imshow(img[:,:,0],cmap='gray')
plt.show()
plt.imshow(img[:,:,1],cmap='gray')
plt.show()
plt.imshow(img[:,:,2],cmap='gray')
plt.show()

Obrázky sú defualtne v uint8, teda prirodzené čísla od 0 do 255. Niekedy chceme dostať tieto hodnoty do floatov a vtedy sa štandardne očakávajú hodnoty medzi 0 a 1.

In [ ]:
img_f = img / 255
print(img_f.dtype)
plt.imshow(img_f[:,:,::-1])
plt.show()

S obrazom môžeme jednoducho manipulovať. Napríklad zosíliť jeden kanál.

In [ ]:
img[:,:,0] += 20
plt.imshow(img[:,:,::-1])
plt.show()

## Tensorflow

Teraz si ukážeme prácu s tenforflowom. V prvej časti si ukážeme workflow tréningu siete a v druhej časti si ukážeme ako sa dá pracovať s objektovými detektormi.


### Tréning neurónovej siete

Teraz si ukážeme ako funguje trénovanie neurónovej siete. Sieť nebudeme trénovať priamo pomocou tensroflowu ale využijeme jeho modul keras, ktorý celý proces zjednodušuje.

Na tréning použijeme dataset MNIST, ktorý obsahuje ručne písané číslice.



In [ ]:
from tensorflow import keras
from keras.datasets import mnist

(x, y), (x_test, y_test) = mnist.load_data()
x = np.reshape(x, (-1, 28, 28, 1)) / 255
x_test = np.reshape(x_test, (-1, 28, 28, 1)) / 255

print("Obrazky mame nacitane v tvare {}".format(x.shape))
plt.imshow(x[0, :, :, 0], cmap='gray')
plt.show()

print("Povodne maju tvar: {} a jeden element vyzera takto: {}".format(y.shape, y[0]))
y = keras.utils.to_categorical(y)
print("Po uprave data maju tvar: {} a jeden element vyzera takto: {}".format(y.shape, y[0]))
y_test = keras.utils.to_categorical(y_test)


Pre tréning si najprv vytvoríme tzv. model, ktorý budeme trénovať. Bude nám stačit tzv sekvenčný model do ktorého môžeme postupne pridávať vrstvy a tie sa budú vždy na obrázok v tom istom poradí aplikovať.

Naša sieť bude mať zopár konvolučných vrstiev po ktorých bude následovať pooling. Toto pár krát zopakujeme až na konci spravíme globálny pooling. Globálny pooling zredukuje rozmery obrázka a ostane nám tak príznakový vektor. Následne pridáme ešte do modelu zopár plne prepojených vrstiev na klasifikáciu. Na konci budeme mať 10 neurónov s aktivačnou funkciou softmax.

In [ ]:
model = keras.models.Sequential()
model.add(keras.layers.Conv2D(32, 3, activation='relu', input_shape=(28, 28, 1)))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(64, 3, activation='relu'))
model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
model.add(keras.layers.Conv2D(128, 3, activation='relu'))
#model.add(keras.layers.MaxPooling2D(pool_size=(2, 2)))
#model.add(keras.layers.Conv2D(256, 3, activation='relu'))

model.add(keras.layers.GlobalAveragePooling2D())

model.add(keras.layers.Dense(128, activation='relu'))
model.add(keras.layers.Dense(64, activation='relu'))
model.add(keras.layers.Dense(10, activation='softmax'))

print(model.summary())

Na to aby sme model natrénovali potrebujeme tzv. cenovú funkciu. Keďže máme 10 kategórií vyberieme categorical crossentropy. Takisto potrebujeme aj vybrať optimalizačný algoritmus. Vyberieme si Adam. Pri kompilácii si ešte vyberieme, metriku ktorá sa nám bude zobrazovať počas trénovanie. Táto metrika nemá vplyv na tréning ale je vhodné použíť nejakú, ktorú dokážeme interpretovať.

In [ ]:
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Tréning je potom jednoduchý a realizujeme ho pomocou model.fit

Pri tréningu si trénovacie dáta rozdelíme na trénovacie a validačné. Validačné sa používajú na to aby sme overili, že tréning prebieha dobre (nenastala nejaká zásadná chyba, alebo sme sa nedostali do bodu preučenia) a aby sme vedeli nastavovať tzv. hyperparametre tréningu. Tréning sa deje po dávkach (batch) postupnými zmenami v parametroch siete. Jedno prejdenie trénovacích obrázkov nazývame epocha. Na konci každej epochy sa model spustí na validačných dátach, ale bez toho aby sa sieť trénovala.

In [ ]:
model.fit(x[:50000], y[:50000], validation_data=(x[50000:], y[50000:]), batch_size=32, epochs=3, shuffle=True)

Model môžeme vyhodnotiť na testovacích dátach buď rovno naraz, alebo si ho môžeme púštať postupne.

In [ ]:
model.evaluate(x_test, y_test)

In [ ]:
for i in range(10):
  img = x_test[i]
  plt.imshow(img[:, :, 0], cmap='gray')
  plt.show()
  pred = model.predict(img[np.newaxis, :, : ,:])
  print(pred)
  print(np.argmax(pred[0]))

### Detekcia chodcov

Prvá úloha bude na detekciu chodcov. Budeme pracovať s rovnakým datasetom ako na 9. cvičení. Teraz ho stiahneme. A zobrazíme jeden obrázok.

In [ ]:
!wget https://github.com/kocurvik/edu/raw/master/PV/materialy/cv09.zip
!unzip cv09.zip

img = cv2.imread('pedxing-seq1/00002071.jpg')
plt.imshow(img[:, :, ::-1])

Model neurónovej siete si stiahneme jednoducho vďaka tensorflow hub. Stiahneme si objektový detektor CenterNet: https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512/1

In [ ]:
import tensorflow_hub as hub
detector = hub.load("https://tfhub.dev/tensorflow/centernet/resnet50v2_512x512/1")

Sieť očakáva na vstupe obrázok v tvare 1 x výška x šírka x 3 v uint8 v rozsahu 0 .. 255. Sieť aplikujeme tak, že zavoláme detector na obrázok. Výstupom je dict s rôznymi hodnotami. Detection scores určujú pravdepodobnosti, že detegovaný objekt je detegovaný správne. detection_boxes obsahuje bounding boxy v relatívnych súradniciach (0, 1) v poradí ymin, xmin, ymax, xmax. detection_classes označujú triedy detekcií podľa datasetu MS COCO: https://gist.github.com/AruniRC/7b3dadd004da04c80198557db5da4bda

Nasledujúci kód spustí neurónovú sieť a zobrazí jej výstup. Chodcov zobrazí zeleným štvoruholníkom a všetky ostatné objekty červeným.

In [ ]:
img = cv2.imread('pedxing-seq1/00002150.jpg')
detector_output = detector(img[np.newaxis, :, :, :])

print(detector_output)

scores = detector_output['detection_scores'].numpy()[0]
classes = detector_output['detection_classes'].numpy()[0]
boxes = detector_output['detection_boxes'].numpy()[0]

width = img.shape[1]
height = img.shape[0]

for i in range(len(scores)):
  if scores[i] < 0.3:
    break

  if classes[i] == 1:
    img = cv2.rectangle(img, (int(width * boxes[i, 1]), int(height * boxes[i, 0])), (int(width * boxes[i, 3]), int(height * boxes[i, 2])), (0, 255, 0), thickness=3)
  else:
    img = cv2.rectangle(img, (int(width * boxes[i, 1]), int(height * boxes[i, 0])), (int(width * boxes[i, 3]), int(height * boxes[i, 2])), (0, 0, 255), thickness=3)

plt.imshow(img[:, :, ::-1])
plt.show()
